In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegressionCV
from sklearn.decomposition import PCA

In [2]:
df_train = pd.read_parquet("../input/amex-data-integer-dtypes-parquet-format/train.parquet")

In [3]:
cat_col = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
nonsig_cols = ["D_65", "S_23", "D_135", "S_24", "B_26", "D_143", "B_40", "D_141", "B_19",
              "S_22", "D_137", "R_7", "D_71", "B_2", "B_27", "D_69", "R_20", "D_83", "D_75",
              "B_36", "R_25", "D_139", "R_28", "R_10", "R_24", "D_119", "R_4", "B_10"]

In [4]:
y = pd.read_csv("../input/amex-default-prediction/train_labels.csv")
df_train = pd.merge(df_train, y, on = "customer_ID")
df_train = df_train.groupby("customer_ID").head(1)
df_date = df_train.pop("S_2")
del df_date
df_train = df_train.set_index("customer_ID", drop = True).sort_index()


In [5]:
y = df_train.pop("target")
features = list(set(df_train.columns) - set(cat_col) - set(nonsig_cols)) 
X = df_train[features]
#cat_dummies = pd.get_dummies(df_train[cat_col])
#X = pd.concat([X, cat_dummies], axis = "columns")
#del df_train

In [6]:
import xgboost as xgb
from sklearn.model_selection import KFold
model = xgb.XGBClassifier(objective= "binary:logistic", seed = 42)
model.fit(X, y, verbose = True, early_stopping_rounds = 20, eval_metric = "aucpr",
         eval_set = [(X, y)])

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  UserWarning,
/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:797: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  UserWarning,


[0]	validation_0-aucpr:0.70443
[1]	validation_0-aucpr:0.71852
[2]	validation_0-aucpr:0.72480
[3]	validation_0-aucpr:0.73011
[4]	validation_0-aucpr:0.73493
[5]	validation_0-aucpr:0.73785
[6]	validation_0-aucpr:0.74095
[7]	validation_0-aucpr:0.74392
[8]	validation_0-aucpr:0.74632
[9]	validation_0-aucpr:0.74872
[10]	validation_0-aucpr:0.75074
[11]	validation_0-aucpr:0.75256
[12]	validation_0-aucpr:0.75435
[13]	validation_0-aucpr:0.75615
[14]	validation_0-aucpr:0.75779
[15]	validation_0-aucpr:0.75940
[16]	validation_0-aucpr:0.76103
[17]	validation_0-aucpr:0.76288
[18]	validation_0-aucpr:0.76474
[19]	validation_0-aucpr:0.76593
[20]	validation_0-aucpr:0.76771
[21]	validation_0-aucpr:0.76898
[22]	validation_0-aucpr:0.76987
[23]	validation_0-aucpr:0.77141
[24]	validation_0-aucpr:0.77278
[25]	validation_0-aucpr:0.77399
[26]	validation_0-aucpr:0.77519
[27]	validation_0-aucpr:0.77644
[28]	validation_0-aucpr:0.77778
[29]	validation_0-aucpr:0.77831
[30]	validation_0-aucpr:0.77870
[31]	validation_0-

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=42,
              reg_alpha=0, reg_lambda=1, ...)

In [7]:
test_df = (pd.read_parquet("../input/amex-data-integer-dtypes-parquet-format/test.parquet")
            .groupby("customer_ID")
           .tail(1)
           .set_index("customer_ID", drop = True)
           .sort_index())

In [8]:
test_df = test_df[features]
y_pred = model.predict_proba(test_df)
sub = pd.read_csv("../input/amex-default-prediction/sample_submission.csv")
sub = sub.sort_values("customer_ID")

In [9]:
sub["prediction"] = y_pred[:, 1]
sub.to_csv("submission.csv", index = False)